## Train your first LM model with Amazon SageMaker

### Sentiment Analysis with `DistilBERT` and `imdb` dataset

1. [Introduction](#Introduction)  
2. [Environment and Permissions](#Environment-and-Permissions)
3. [Preprocess - Tokenization of the dataset](#Preprocessing)   
4. [Creating an Estimator and start a training job](#Creating-an-Estimator-and-start-a-training-job)  
5. [Deploying the endpoint](#Deploying-the-endpoint)  

# Introduction

Welcome to our end-to-end binary Text-Classification example. In this demo, we will use the Hugging Face `transformers` and `datasets` library and the SageMaker SDK to launch a SageMaker Training job and fine-tune a pre-trained transformer for binary text classification. In particular, we will use the pre-trained DistilBERT model and fine-tune it using the `imdb` dataset. To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on. 


_**NOTE: You can run this demo in Sagemaker Studio, your local machine or Sagemaker Notebook Instances**. This notebook has been tested in SageMaker Studio with a PyTorch 1.13 Python 3.9 CPU Optimized Kernel_ 

## Environment and Permissions 

In [2]:
!pip install datasets
!pip install -U sagemaker
!pip install -U transformers
!pip install -U accelerate

  Using cached datasets-2.14.3-py3-none-any.whl (519 kB)
  Using cached xxhash-3.3.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (193 kB)
  Using cached aiohttp-3.8.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached multidict-6.0.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached frozenlist-1.4.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (228 kB)
  Using cached yarl-1.9.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (269 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
  Using cached sagemaker-2.175.0-py2.py3-none-any.whl
  Using cached PyYAML-6.0

In [3]:
import sagemaker
import boto3
import sagemaker.huggingface

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::897205850961:role/service-role/AmazonSageMaker-ExecutionRole-20230414T104809
sagemaker bucket: sagemaker-eu-central-1-897205850961
sagemaker session region: eu-central-1


## Visualizing our data
We are using the `datasets` library to download the `imdb` [dataset](https://huggingface.co/datasets/imdb). The dataset consists of 25,000 highly polar movie reviews for training, and 25,000 for testing.
Let's see how our dataset looks like

In [6]:
from datasets import load_dataset

train_dataset, test_dataset = load_dataset("imdb", split=["train", "test"])

In [7]:
train_dataset, test_dataset

(Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }))

In [8]:
train_dataset[10]

{'text': 'It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is a

The dataset includes the 'text' field, which is the free text review, comment about the movie, and the 'label' field, which is a binary variable coded with value 0 for negative review and value 1 for positive ones. 

# Preprocessing

Before you can train a model on a dataset, it needs to be preprocessed into the expected model input format. Whether your data is text, images, or audio, they need to be converted and assembled into batches of tensors.
Text, use a [Tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer) to convert text into a sequence of tokens, create a numerical representation of the tokens, and assemble them into tensors.

## Tokenization 

In [9]:
from datasets import load_dataset
from transformers import AutoTokenizer

tokenizer_name = 'distilbert-base-uncased'
dataset_name = 'imdb'
# s3 key prefix for the data
s3_prefix = 'samples/datasets/imdb'

# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# load dataset
train_dataset, test_dataset = load_dataset(dataset_name, split=['train', 'test'])
test_dataset = test_dataset.shuffle().select(range(10000)) # smaller the size for test dataset to 10k 

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set format for pytorch
train_dataset =  train_dataset.rename_column("label", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataset = train_dataset.remove_columns("text")
test_dataset = test_dataset.remove_columns("text")

Map: 100%|██████████| 10000/10000 [00:11<00:00, 869.47 examples/s]


## Uploading data to `sagemaker_session_bucket`

After we processed the `datasets` we are going to use the new `FileSystem` [integration](https://huggingface.co/docs/datasets/filesystems.html) to upload our dataset to S3.

In [10]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path)

Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:01<00:00, 9883.54 examples/s] 


## Creating an Estimator and starting a training job

From all supported models from Hugging Face, we selected https://huggingface.co/distilbert-base-uncased. It is light ("distilled") version of the BERT model, introduced in [this paper](https://arxiv.org/abs/1910.01108), where authors claim to have reduced the size of the model by 40% compared to BERT-base, while retaining 97%
of its language understanding capabilities and being 60% faster. Size reduction and faster execution helps to reduce the footprint of the instance required for training and inference, reducing trainig time.
According to the model card in hugging face, the raw model can be used for next sentence prediction but it's mostly intended to be fine-tuned on a downstream task like as sequence classification, token classification or question answering.
In our case we will fine-tune the model to binary classify the sentence as brining positive or negative sentiment.   

### Language Model Training (Model Fine-Tuning)
Language Models size ranges from hundreds of million to hundreds of billion of parameters. For example BERT-base consists of 110M parameters. As training such models can take weeks, months ore much more, it is common practise to start from a pre-trained model and tune the network parameters (called model fine-tuning) using domain specific datasets with supervised learning. 

This is what we will do in the following. As you will see, the training (fine-tuning) on the emotion data set, takes just about 12 minutes. 

In [12]:
from sagemaker.huggingface import HuggingFace

training_input_path = "s3://{}/{}/train".format(sess.default_bucket(), s3_prefix)
test_input_path = "s3://{}/{}/test".format(sess.default_bucket(), s3_prefix)

# hyperparameters, which are passed into the training job
hyperparameters = {
    "epochs": 1,
    "train_batch_size": 32,
    "model_name": "distilbert-base-uncased",
    "learning_rate": 0.00003,
}

In [13]:
huggingface_estimator = HuggingFace(
    entry_point="train_script.py",
    source_dir="./scripts",
    instance_type="ml.g4dn.12xlarge",
    instance_count=1,
    role=role,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",
    hyperparameters=hyperparameters,
)

In [14]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({"train": training_input_path, "test": test_input_path})

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-08-07-14-51-17-633


2023-08-07 14:51:18 Starting - Starting the training job...
2023-08-07 14:51:33 Starting - Preparing the instances for training.........
2023-08-07 14:53:12 Downloading - Downloading input data
2023-08-07 14:53:12 Training - Downloading the training image...........................
2023-08-07 14:57:23 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-08-07 14:58:08,582 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-08-07 14:58:08,617 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-08-07 14:58:08,628 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-08-07 14:58:08,630 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-08-07 14:58:08,871 sagemaker-training-toolkit INFO     No Neurons de

While waiting for training to complete, which will take about 12 minutes, you can move to the SageMaker Console, click on Training -> Training Jobs and you will see your training job (InProgress Status) that you started calling the fit method in the cell above.
When the training is completed you can see your training Job in Completed Status. Clicking on the job name you are directed to a page reporting several details of your training job, in particular at bottom of the page you can find the location of the model artifact you just created. 

## Deploying the endpoint

To deploy our endpoint, we call `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [15]:
predictor = huggingface_estimator.deploy(initial_instance_count=1, instance_type="ml.g4dn.xlarge")

INFO:sagemaker:Creating model with name: huggingface-pytorch-training-2023-08-07-15-09-49-895
INFO:sagemaker:Creating endpoint-config with name huggingface-pytorch-training-2023-08-07-15-09-49-895
INFO:sagemaker:Creating endpoint with name huggingface-pytorch-training-2023-08-07-15-09-49-895


--------!

Then, we use the returned predictor object to call the endpoint.

In [16]:
sentiment_input = {
    "inputs": "This is the best movie ever made in history, an absolute sculpted work of art that depicts every emotion of human existence, from suffering, to courage to love, in front of the background of political astuteness and socio-hierarchal analysis."
}

predictor.predict(sentiment_input)

[{'label': 'LABEL_1', 'score': 0.9557088017463684}]

In [17]:
sentiment_input = {
    "inputs": "Another bloated film that gets all the history wrong, turns all of the characters into stick figures and makes piles of money for the star."
}

predictor.predict(sentiment_input)

[{'label': 'LABEL_0', 'score': 0.9047076106071472}]

Finally, we delete the endpoint again.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()